# Data
---

In [37]:
import os
import json
import heapq
import pickle
import random
import multiprocessing

import spacy
from tqdm.notebook import tqdm

from valerie.utils import get_logger
from valerie.preprocessing import extract_words_from_url, clean_text
from valerie.scoring import validate_predictions_phase2, compute_score_phase2
from valerie.modeling import SequenceClassificationModel, SequenceClassificationDataset, SequenceClassificationExample

In [2]:
nlp = spacy.load("en_core_web_lg")

In [3]:
_logger = get_logger()

In [4]:
with open("data/phase2-3/processed/responses.pkl", "rb") as fi:
    responses = pickle.load(fi)

In [5]:
responses = responses[:1000]

In [6]:
len(responses)

1000

In [7]:
def compute_responses_score(results, claims_dict):
    predictions = {}
    perfect_predictions = {}
    labels = {}

    for k, hits in results.items():
        claim = claims_dict[k]
        labels[claim.id] = claim.to_dict()
        
        hits = sorted(hits, key=lambda x: x[1], reverse=True) # sort by score
        predictions[claim.id] = {
            "label": claim.label,
            "explanation": "",
            "related_articles": {
                i + 1: x
                for i, x in enumerate([v[0] for v in hits[:2]])
            }
        }
        perfect_predictions[claim.id] = {
            "label": claim.label,
            "explanation": "",
            "related_articles": {
                i + 1: x
                for i, x in enumerate([v[0] for v in hits if v[0] in claim.related_articles.values()][:2])
            }
        }

    validate_predictions_phase2(predictions)
    score = compute_score_phase2(labels, predictions)
    validate_predictions_phase2(perfect_predictions)
    perfect_score = compute_score_phase2(labels, perfect_predictions)
    return {
        "perfect_rerank_score": perfect_score["score"],
        "perfect_rerank_error": perfect_score["error"],
        "api_score": score["score"],
        "api_error": score["error"],
    }

In [8]:
def create_text_a(claim):
    text_a = claim.claim
    text_a += " "
    text_a += claim.claimant if claim.claimant else "no claimant"
    text_a += " "
    text_a += claim.date.split()[0].split("T")[0] if claim.date else "no date"
    return clean_text(text_a)

def create_text_b_content(article):
    text_b = ""
    if article.source:
        text_b += article.source + ". "
    if article.title:
        text_b += article.title + ". "
    if article.url:
        url_words = extract_words_from_url(article.url)
        if url_words:
            text_b += " ".join(url_words) + ". "
    if article.content:
        text_b += article.content
    return clean_text(text_b)

# Run Spacy on Data

### Claims

In [9]:
misses = 0
claims_list = []
for res in tqdm(responses):
    if not res["res"]:
        misses += 1
        continue
    claim = res["claim"]
    claim.text_a = create_text_a(claim)
    claim.res = res
    claim.support = {}
    claims_list.append(claim)

In [10]:
misses

0

In [11]:
claims_texts = [claim.text_a for claim in claims_list]

In [12]:
claims_docs = [doc for doc in tqdm(nlp.pipe(claims_texts, n_process=16, disable=["textcat", "tagger", "parser", "ner"]), total=len(claims_texts))]

In [30]:
claims_dict = {}
for claim, doc in tqdm(zip(claims_list, claims_docs)):
    claim.doc = doc
    claims_dict[claim.index] = claim

In [14]:
len(claims_list)

1000

In [15]:
len(set(claims_list))

1000

### Articles

In [16]:
misses = 0
articles_list = []
for res in tqdm(responses):
    if not res["res"]:
        misses += 1
        continue
    for hit in res["res"]["hits"]["hits"]:
        article = hit["article"]
#         article.text_b = create_text_b_content(article)
        articles_list.append(article)

In [17]:
misses

0

In [18]:
len(articles_list)

26629

In [19]:
len(set(articles_list))

19626

In [20]:
articles_list = list(set(articles_list))

In [21]:
def _text_b_text(article):
    return article, create_text_b_content(article)

articles_texts = {}
pool = multiprocessing.Pool(16)
for article, text_b in tqdm(pool.imap_unordered(_text_b_text, articles_list), total=len(articles_list)):
    articles_texts[article.index] = text_b

In [22]:
for article in articles_list:
    article.text_b = articles_texts[article.index]

In [23]:
articles_texts = [article.text_b for article in tqdm(articles_list)]

In [24]:
articles_docs = [doc for doc in tqdm(nlp.pipe(articles_texts, n_process=16, disable=["textcat", "tagger", "ner"]), total=len(articles_texts))]

In [31]:
articles_dict = {}
for article, doc in tqdm(zip(articles_list, articles_docs)):
    article.doc = doc
    articles_dict[article.index] = article

# Examples
---

In [35]:
def create_text_b_curated(article, claim):
    support = []
    for sent in article.doc.sents:
        support.append({
            "text": sent.text,
            "score": claim.doc.similarity(sent)
        })
    support = heapq.nlargest(16, support, key=lambda x: x["score"])
    claim.support[article.index] = support
    text_b = clean_text(" ".join([s["text"] for s in support]))
    return text_b

In [38]:
examples = []
for claim in tqdm(claims_dict.values()):
    hits_indices = [hit["url"] for hit in claim.res["res"]["hits"]["hits"]]
    hits = [articles_dict[idx] for idx in hits_indices]
    
    related_articles_url_set = set(claim.related_articles.values())

    for article in hits[:16]:
        article.text_b = create_text_b_curated(article, claim)

        examples.append(SequenceClassificationExample(
            guid=claim.index,
            text_a=claim.text_a,
            text_b=article.text_b,
            label=1 if article.url in related_articles_url_set else 0,
            art_id=article.index
        ))

  0%|          | 1/1000 [00:00<02:17,  7.26it/s]/home/jay/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  
100%|██████████| 1000/1000 [02:42<00:00,  6.15it/s]


In [41]:
print(len(claims_dict))
print(len(articles_dict))
print()
print(len(claims_dict)*16)
print(len(examples))

1000
19626

16000
15975


In [43]:
examples[0]

{
  "guid": "Phase2Dataset/6093",
  "text_a": "\"To deprive\" underprivileged children \"of a meat-based protein during school lunch is most likely depriving them of their only source of protein for the day.\" Todd Staples 2014-09-08",
  "text_b": "For as little as $6 a month, you can help support our independent journalism \u2014 and enjoy special benefits and status as a Times of Israel Community member! But we would like to invite readers who can afford to do so, and for whom The Times of Israel has become important, to help support our journalism by joining Join now and for as little as $6 a month you can both help ensure our ongoing investment in quality journalism, and enjoy special status and benefits as a Times of Israel Community member. The protein is known to have restorative qualities, but the problem is how to get it to the brain, she said. Thus, the neural growth factor proteins, if given in drug form, do not pass through the barrier, and even if they could, they would no

In [49]:
list(claims_dict.values())[-1].claim

'"U.S. troops are getting only 4 hours of Ebola training before deployment to Liberia."'

In [ ]:
print(json.dumps(list(claims_dict.values())[-1].support, indent=2))

In [50]:
print(len(responses)*16)
print(len(examples))

16000
15975


# Predict
---

In [51]:
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "dryrun"
os.environ["WANDB_WATCH"] = "false"

In [52]:
# "castorini/monot5-base-msmarco"
# "castorini/monobert-large-msmarco"
# "nboost/pt-bert-large-msmarco"]:
pretrained_model_name_or_path = "castorini/monobert-large-msmarco"

In [53]:
model = SequenceClassificationModel.from_pretrained(pretrained_model_name_or_path)

[2020-07-12 14:48:34,942] INFO:transformers.configuration_utils: loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/castorini/monobert-large-msmarco/config.json from cache at /home/jay/.cache/torch/transformers/643500d870067d59f219f7b5652919267c01bfa98024e2e74f53b28c1b6aff2b.4c88e2dec8f8b017f319f6db2b157fee632c0860d9422e4851bd0d6999f9ce38
[2020-07-12 14:48:34,944] INFO:transformers.configuration_utils: Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

[2020-07-12 14:48:34,945] INFO:transformers.tokenization_utils_base: Model name 'castorini/monobert-large-msmarco' not f

In [54]:
examples_dataset = model.create_dataset(examples, nproc=16)

[2020-07-12 14:48:55,655] INFO:valerie.modeling: ... converting examples to features ...


In [55]:
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3,4,5,6,7"
!echo $CUDA_VISIBLE_DEVICES

0,1,2,3,4,5,6,7


In [56]:
predict_output = model.predict(examples_dataset, predict_batch_size=256)

[2020-07-12 14:49:12,517] INFO:transformers.training_args: PyTorch: setting up devices
[2020-07-12 14:49:16,210] INFO:transformers.trainer: Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
[2020-07-12 14:49:16,217] INFO:transformers.trainer: ***** Running Prediction *****
[2020-07-12 14:49:16,218] INFO:transformers.trainer:   Num examples = 15975
[2020-07-12 14:49:16,220] INFO:transformers.trainer:   Batch size = 2048


In [57]:
claims_dict = {res["claim"].index: res["claim"] for res in responses if res["res"]}
api_scores_dict = {
    res["claim"].index: {
        hit["article"].index: hit["score"] for hit in res["res"]["hits"]["hits"]
    }
    for res in responses
    if res["res"]
}


rerank_just_api_responses = {
    res["claim"].index: [
        (hit["article"].index, hit["score"]) for hit in res["res"]["hits"]["hits"]
    ]
    for res in responses
    if res["res"]
}

rerank_just_trans_responses = {res["claim"].index: [] for res in responses if res["res"]}

rerank_both_responses = {
    res["claim"].index: []
    for res in responses
    if res["res"]
}

for example, proba in tqdm(zip(examples, predict_output.predictions)):
    proba = float(proba[1]) # get probability that the article is related

    rerank_just_trans_responses[example.guid].append((example.art_id, proba))
    rerank_both_responses[example.guid].append((example.art_id, proba + api_scores_dict[example.guid][example.art_id]))
    
print('api')
print(json.dumps(compute_responses_score(rerank_just_api_responses, claims_dict), indent=2))
print()
print('trans')
print(json.dumps(compute_responses_score(rerank_just_trans_responses, claims_dict), indent=2))
print()
print('both')
print(json.dumps(compute_responses_score(rerank_both_responses, claims_dict), indent=2))
print()
print()
print()

15975it [00:00, 381521.82it/s]

api
{
  "perfect_rerank_score": 0.8630615319780865,
  "perfect_rerank_error": "'None'",
  "api_score": 0.448855518144285,
  "api_error": "'None'"
}

trans
{
  "perfect_rerank_score": 0.7876843776993298,
  "perfect_rerank_error": "'None'",
  "api_score": 0.5356888355106825,
  "api_error": "'None'"
}

both
{
  "perfect_rerank_score": 0.7876843776993298,
  "perfect_rerank_error": "'None'",
  "api_score": 0.4956699996625343,
  "api_error": "'None'"
}





# Results
---

In [58]:
compute_responses_score(rerank_just_api_responses, claims_dict)

{'perfect_rerank_score': 0.8630615319780865,
 'perfect_rerank_error': "'None'",
 'api_score': 0.448855518144285,
 'api_error': "'None'"}

In [59]:
compute_responses_score(rerank_just_trans_responses, claims_dict)

{'perfect_rerank_score': 0.7876843776993298,
 'perfect_rerank_error': "'None'",
 'api_score': 0.5356888355106825,
 'api_error': "'None'"}

In [60]:
compute_responses_score(rerank_both_responses, claims_dict)

{'perfect_rerank_score': 0.7876843776993298,
 'perfect_rerank_error': "'None'",
 'api_score': 0.4956699996625343,
 'api_error': "'None'"}